#Preliminaries

In [1]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
#!tar xf spark-3.2.0-bin-hadoop3.2.tgz
#!pip install -q findspark

In [2]:
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/07 05:32:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/07 05:32:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
import pyspark.sql
from pyspark.sql import Row
from pyspark.sql.types import *
import json

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
import numpy as np
np.set_printoptions(threshold=np.inf,linewidth=np.inf)
import string
import math
import statistics
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [8]:
 homedir = '/home/bcavna/Documents'

#Get Wind Speed Normalization Factors

Get wind speed values that were collected during data pre-processing

In [9]:
wsPath =  homedir + '/ERA5/processed/normvals/WS/*'
print(wsPath)

/home/bcavna/Documents/ERA5/processed/normvals/WS/*


In [10]:
rawdataDF = spark.read.csv(wsPath, header="true")
rawdataDF.createOrReplaceTempView("ws")
rawdataDF.show(5)

+---+------+--------+
|_c0|WS_SUM|WS_COUNT|
+---+------+--------+
|  0|   0.0|    5269|
|  1|   1.0|   55885|
|  2|   2.0|  140801|
|  3|   3.0|  190289|
|  4|   4.0|  211536|
+---+------+--------+
only showing top 5 rows



23/12/07 05:32:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , WS_SUM, WS_COUNT
 Schema: _c0, WS_SUM, WS_COUNT
Expected: _c0 but found: 
CSV file: file:///home/bcavna/Documents/ERA5/processed/normvals/WS/WS_FREQ_201512.csv


In [11]:
wstotDF = spark.sql("SELECT WS_SUM AS WS, SUM(WS_COUNT) AS WS_COUNT FROM ws GROUP BY WS_SUM ORDER BY WS_SUM*1")
wstotDF.show(10)

+---+-----------+
| WS|   WS_COUNT|
+---+-----------+
|0.0|  3309654.0|
|1.0|2.6855975E7|
|2.0|5.3327146E7|
|3.0|6.5258814E7|
|4.0|7.1220865E7|
|5.0|7.3904752E7|
|6.0|7.4224084E7|
|7.0|7.3079824E7|
|8.0|7.0961494E7|
|9.0|6.8234437E7|
+---+-----------+
only showing top 10 rows



Calculate cdf values to use for normalization

In [12]:
wslist = wstotDF.rdd.map(lambda x: (float(x[0]),x[1])).collect()

In [13]:
wstot = wstotDF.rdd.map(lambda x: x[1]).collect()
print(wstot)

[3309654.0, 26855975.0, 53327146.0, 65258814.0, 71220865.0, 73904752.0, 74224084.0, 73079824.0, 70961494.0, 68234437.0, 65161840.0, 62007385.0, 58818582.0, 55621746.0, 52433374.0, 49366346.0, 46472561.0, 43703740.0, 41065437.0, 38614380.0, 36329949.0, 34141196.0, 32064283.0, 30162083.0, 28322858.0, 26606968.0, 25022830.0, 23568759.0, 22246812.0, 21000997.0, 19806872.0, 18697785.0, 17662202.0, 16699702.0, 15769121.0, 14922754.0, 14101110.0, 13330642.0, 12576793.0, 11875758.0, 11219556.0, 10578911.0, 9984391.0, 9401784.0, 8860559.0, 8319084.0, 7823481.0, 7349739.0, 6914760.0, 6514202.0, 6111124.0, 5728029.0, 5375002.0, 5049710.0, 4744623.0, 4449985.0, 4174200.0, 3919789.0, 3670183.0, 3431583.0, 3208009.0, 3003913.0, 2808070.0, 2626653.0, 2460518.0, 2304129.0, 2166791.0, 2034953.0, 1914686.0, 1791173.0, 1662670.0, 1543144.0, 1429594.0, 1328474.0, 1233364.0, 1147395.0, 1063802.0, 986877.0, 920071.0, 857423.0, 802694.0, 753482.0, 713159.0, 674712.0, 640441.0, 606752.0, 570109.0, 537028.0, 5

In [14]:
wslist[:5]

[(0.0, 3309654.0),
 (1.0, 26855975.0),
 (2.0, 53327146.0),
 (3.0, 65258814.0),
 (4.0, 71220865.0)]

In [15]:
def getCDF(slist,stot):
  run = 0
  dist = []
  for l in slist:
    run += l[1]
    dist.append([l[0],l[1],round(run/stot,2)])
  return dist

In [16]:
cdf = getCDF(wslist,sum(wstot))
print(cdf[:5])

[[0.0, 3309654.0, 0.0], [1.0, 26855975.0, 0.02], [2.0, 53327146.0, 0.05], [3.0, 65258814.0, 0.09], [4.0, 71220865.0, 0.13]]


In [17]:
headers = ['ws','freq','cdf']
cdfR = sc.parallelize(cdf)
cdfPD = cdfR.toDF(headers).toPandas()

In [18]:
cdfPD.head(10)

,ws,freq,cdf
0,0.0,3309654.0,0.00
1,1.0,26855975.0,0.02
2,2.0,53327146.0,0.05
3,3.0,65258814.0,0.09
4,4.0,71220865.0,0.13
5,5.0,73904752.0,0.17
6,6.0,74224084.0,0.21
7,7.0,73079824.0,0.26
8,8.0,70961494.0,0.30
9,9.0,68234437.0,0.34


In [19]:
wsOut = homedir + '/ERA5/processed/normvals/ws_facs.cvs'
print(wsOut)

/home/bcavna/Documents/ERA5/processed/normvals/ws_facs.cvs


In [20]:
cdfPD.to_csv(wsOut)

#Get Verticle Wind Normalization Factors

In [21]:
myPath2 = homedir + '/ERA5/processed/normvals/VV/*'
print(myPath2)

/home/bcavna/Documents/ERA5/processed/normvals/VV/*


In [22]:
rawdataDF2 = spark.read.csv(myPath2, header="true")
rawdataDF2.createOrReplaceTempView("vv")
rawdataDF2.show(5)

+---+-----+--------+
|_c0|   vv|VV_COUNT|
+---+-----+--------+
|  0|-16.0|       1|
|  1|-15.0|       2|
|  2|-14.0|       1|
|  3|-13.0|       5|
|  4|-12.0|       5|
+---+-----+--------+
only showing top 5 rows



23/12/07 05:32:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , vv, VV_COUNT
 Schema: _c0, vv, VV_COUNT
Expected: _c0 but found: 
CSV file: file:///home/bcavna/Documents/ERA5/processed/normvals/VV/VV_FREQ_201606.csv


In [23]:
vvDistDF = spark.sql("SELECT vv AS vv, SUM(vv_COUNT) AS vv_COUNT FROM vv GROUP BY vv ORDER BY vv*1 ")
vvDistDF.show(500)

+-----+-------------+
|   vv|     vv_COUNT|
+-----+-------------+
|-16.0|          1.0|
|-15.0|          2.0|
|-14.0|          8.0|
|-13.0|         21.0|
|-12.0|         23.0|
|-11.0|         70.0|
|-10.0|        152.0|
| -9.0|        367.0|
| -8.0|        945.0|
| -7.0|       2817.0|
| -6.0|       9198.0|
| -5.0|      30948.0|
| -4.0|     111625.0|
| -3.0|     480512.0|
| -2.0|    2790134.0|
| -1.0|  3.4649545E7|
|  0.0|1.946415537E9|
|  1.0|  1.4322364E7|
|  2.0|     185036.0|
|  3.0|      10668.0|
|  4.0|        656.0|
|  5.0|         35.0|
+-----+-------------+



In [24]:
vvtotDF = spark.sql("SELECT SUM(vv*vv_COUNT) AS sumprod, SUM(vv_COUNT) AS TOT_COUNT  FROM vv")
vvtotDF.show(500)

+------------+-------------+
|     sumprod|    TOT_COUNT|
+------------+-------------+
|-2.7634117E7|1.999010664E9|
+------------+-------------+



In [25]:
vvAVG = vvtotDF.rdd.map(lambda x: x[0]/x[1]).collect()

In [26]:
print(vvAVG)

[-0.013823896739352262]


In [27]:
vvCount = vvtotDF.rdd.map(lambda x: x[1]).collect()

In [28]:
print(vvCount)

[1999010664.0]


In [29]:
def vvStdv(xlist):
  outList = []
  for x in xlist:
    n = (float(x[0])-vvAVG[0])**2
    t = n*float(x[1])
    s = t/(vvCount[0]-1)
    outList.append(s)
  return sum(outList)

In [30]:
vvList = vvDistDF.rdd.map(lambda x: list([float(x[0]),float(x[1])])).collect()
print(vvList)

[[-16.0, 1.0], [-15.0, 2.0], [-14.0, 8.0], [-13.0, 21.0], [-12.0, 23.0], [-11.0, 70.0], [-10.0, 152.0], [-9.0, 367.0], [-8.0, 945.0], [-7.0, 2817.0], [-6.0, 9198.0], [-5.0, 30948.0], [-4.0, 111625.0], [-3.0, 480512.0], [-2.0, 2790134.0], [-1.0, 34649545.0], [0.0, 1946415537.0], [1.0, 14322364.0], [2.0, 185036.0], [3.0, 10668.0], [4.0, 656.0], [5.0, 35.0]]


In [31]:
vvStdv(vvList)

0.03405406638399148

In [32]:
vvFacs = [round(float(vvAVG[0]),2),round(math.sqrt(vvStdv(vvList)),2)]
print(vvFacs)

[-0.01, 0.18]


In [33]:
mat = np.zeros((1,2),float)
mat

array([[0., 0.]])

In [34]:
mat[0,0] = vvFacs[0]
mat[0,1] = vvFacs[1]
mat

array([[-0.01,  0.18]])

In [35]:
vvPD = pd.DataFrame(data=mat,columns=['vvMean','vvStdv'])
vvPD.head()

,vvMean,vvStdv
0,-0.01,0.18


In [36]:
vvOut = homedir + '/ERA5/processed/normvals/vv_facs.cvs'
print(vvOut)

/home/bcavna/Documents/ERA5/processed/normvals/vv_facs.cvs


In [37]:
vvPD.to_csv(vvOut)